In [17]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit
from pyspark.sql.window import Window
import duckdb

In [2]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/16 16:16:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [37]:
# compravenda_sup
DF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:formatted_zone/compravenda_sup.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", "SELECT * FROM RDD") \
    .load()

# verificar duplicats
print(f"Hi ha {DF.count() - DF.distinct().count()} valors duplicats al DataFrame 'formatted_zone/compravenda_sup.db' de {DF.count()} valors totals")

# substituir "--" por NA
DF = DF.withColumn('Nombre', F.when(col('Nombre') == '--', None).otherwise(col('Nombre')))
# verificar NA
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs')
# imprimir toda la informació que té un NA
valorsnull = DF.filter(DF.Nombre.isNull()).show()

# passar a numeric
DF = DF.withColumn('Nombre', DF['Nombre'].cast('float'))

# Imputar NA de Nombre amb la mitjana de "Nombre" per "Nom_barri"
mean_nombre_nom_barri = DF.groupBy('Nom_barri').agg(F.mean('Nombre').alias('mean_nombre_nom_barri'))

DF = DF.join(mean_nombre_nom_barri, on='Nom_barri', how='left')
DF = DF.withColumn('Nombre', F.when(DF['Nombre'].isNull(), DF['mean_nombre_nom_barri']).otherwise(DF['Nombre']))

DF.show()

# escalar
min_max = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:formatted_zone/rent_price.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT min(Price), max(Price) FROM RDD") \
  .load()
  
  

Hi ha 0 valors duplicats al DataFrame 'formatted_zone/compravenda_sup.db' de 876 valors totals
Hi ha 290 NAs
+----+---------+--------------+-------------------+----------+--------------------+----------------------------------+------+
| Any|Trimestre|Codi_Districte|      Nom_Districte|Codi_Barri|           Nom_Barri|Superfície_mitjana_(m2_construïts)|Nombre|
+----+---------+--------------+-------------------+----------+--------------------+----------------------------------+------+
|2023|        1|             7|     Horta-Guinardó|        42|            la Clota|                             Total|  NULL|
|2023|        1|             8|         Nou Barris|        56|            Vallbona|                             Total|  NULL|
|2023|        1|             1|       Ciutat Vella|         1|            el Raval|              Habitatge nou lliure|  NULL|
|2023|        1|             2|           Eixample|         5|       el Fort Pienc|              Habitatge nou lliure|  NULL|
|2023|   